# Note this file is obsolete (the code in package lyman_alpha_reconstruction was previously written in Neural_Net_Library.ipynb and Master.ipynb)

In [2]:
import os
import torch
import torch.distributions as tdist
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = (20,10)
torch.set_default_tensor_type('torch.cuda.FloatTensor')

from datetime import datetime
import numpy as np
from numpy import log, sqrt, exp
import imageio
from scipy.io import FortranFile
from scipy.stats import pearsonr
from scipy.interpolate import CubicSpline
from scipy.ndimage import gaussian_filter1d

nlos = 65536
65536 512 3.0 42807.71484375 400.0
len(taured)= 33554432
nlos,npix= 65536 512
shape of taured= (65536, 1, 512)


## Data loading and processing

In [ ]:
f = FortranFile('C:/Users/Lawrence Huang/Desktop/Research/newz3taured.dat', 'r')
nlos=int(np.asscalar(f.read_ints()))
print("nlos = %d" %nlos)
npix=int(np.asscalar(f.read_ints()))
zred=np.asscalar(f.read_record('f4'))
blenkms=np.asscalar(f.read_record('f4'))
blen=np.asscalar(f.read_record('f4'))*0.001  #back into mpc/h
print(nlos,npix,zred,blenkms,blen)
taured=[]
nstep=1 #skipping through in steps of 1
for i in range(0,nlos,nstep):       
    tauredin=f.read_record('f4')
    taured.extend(tauredin)
f.close()
print('len(taured)=',len(taured))

nlos=int(nlos/nstep)

print ('nlos,npix=',nlos,npix)

taured=np.array(taured)

taured=np.reshape(taured,(nlos,1,npix))
reshapedTaured = np.reshape(taured, (256,256,1,npix))

print('shape of taured=',taured.shape)
# print(reshapedTaured[0,0] == taured[0])
# print(reshapedTaured[1,1] == taured[257])
# print(taured[0,0])

In [3]:
tauValidate = np.reshape(reshapedTaured[0:114,0:114],(114*114,1,512))
tauTest = np.reshape(reshapedTaured[-114:,-114:],(114*114,1,512))
tauTrain = np.concatenate((
               np.reshape(reshapedTaured[:114,114:],(114*(256-114),1,512)), \
               np.reshape(reshapedTaured[114:-114],(256*(256-2*114),1,512)), \
               np.reshape(reshapedTaured[-114:,:-114],(114*(256-114),1,512))))


In [4]:
tauTest = gaussian_filter1d(tauTest, 6, axis=-1, mode="wrap")
tauValidate = gaussian_filter1d(tauValidate, 6, axis=-1, mode="wrap")
tauTrain = gaussian_filter1d(tauTrain, 6, axis=-1, mode="wrap")

In [8]:
mintaured=taured.min()
maxtaured=taured.max()
taured[taured > 0.5e9] = -1.e10

tauTest[tauTest > 0.5e9] = -1.e10
tauValidate[tauValidate > 0.5e9] = -1.e10
tauTrain[tauTrain > 0.5e9] = -1.e10

maxtaured=taured.max()
print (maxtaured)
# set to realistic max:

tauTest[tauTest < -0.5e9] = maxtaured
tauValidate[tauValidate < -0.5e9] = maxtaured
tauTrain[tauTrain < -0.5e9] = maxtaured


flux=np.exp(-1.*taured)
fluxTest = np.exp(-1.*tauTest)
fluxValidate = np.exp(-1.*tauValidate)
fluxTrain = np.exp(-1.*tauTrain)

signalRMS = np.sqrt(np.mean(flux**2))
print("signalRMS", signalRMS)

tauTest = torch.from_numpy(tauTest).cuda()
tauValidate = torch.from_numpy(tauValidate).cuda()
tauTrain = torch.from_numpy(tauTrain).cuda()


fluxTest = torch.from_numpy(fluxTest).cuda()
fluxValidate = torch.from_numpy(fluxValidate).cuda()
fluxTrain = torch.from_numpy(fluxTrain).cuda()

tauredzero=taured[0,0,...]

fluxzero=flux[0,0,...]

plt.plot(tauredzero)
plt.plot(fluxzero)
plt.show()


npHistResult = np.histogram(taured, bins=[0,2,10000])
binProportions = npHistResult[0]/(2**25)
binProportions = torch.from_numpy(binProportions).cuda()

663.7092


TypeError: can't assign a numpy.float32 to a torch.cuda.FloatTensor

In [5]:
del reshapedTaured

In [6]:
noiseDict = {
    "none": 0,
    "low": 1/10,
    "mid": 1/5,
    "high": 1/2.5
}
    
def spin(tensor, midpoint):
    return torch.cat((tensor[midpoint-256:],tensor[:midpoint-256]),dim=0)
    


In [26]:
def spinTest():
    a = torch.arange(512)
    for i in range(512):
        assert(spin(a,i)[256] == i)
#         print("Passed test %d/512" %(i+1))
    print("The spin function works")
    
spinTest()

The spin function works
torch.Size([50])
torch.Size([50, 512])


In [ ]:
class Convolutional_Neural_Net(torch.nn.Module):
    def __init__(self):
        super(Convolutional_Neural_Net, self).__init__()
        self.conv1 = torch.nn.Conv1d(1, 4, 5)
        self.pool1 = torch.nn.MaxPool1d(5,stride = 1)
        self.lin1 = torch.nn.Linear(504,64)
        self.lin2 = torch.nn.Linear(64,8)
        self.predict = torch.nn.Linear(32,1)   # output layer

    def forward(self, x):
        x = F.relu(self.conv1(x))

        x = self.pool1(x)

        x = F.relu(self.lin1(x))

        x = F.relu(self.lin2(x))
        
        x = x.view((-1,1,32))
        
        x = self.predict(x)
        
        return x

In [1]:
def log_predictor(a):
    x = []
    y = []
    a = a.cpu()
    n = len(a)
    for i in range(n):
        if a[i] > 0:
            x.append(i)
            y.append(-log(a[i]))
            
    x.append(x[0] + 512)
    y.append(y[0])
    func = CubicSpline(x,y, bc_type="periodic")
    desiredXs = np.arange(n)
    result = func(desiredXs)
    return result


In [ ]:
def RMSE(pred, act):
    
    pred = torch.Tensor(pred)
    act = torch.Tensor(act)
    result = torch.sqrt(torch.mean((pred-act)**2))
    print(result)
    return result

In [1]:
loss_func = torch.nn.MSELoss()

class NetFactory():
    def __init__(self, noise, printAll = True):
        self.net = Convolutional_Neural_Net().cuda()
            
        self.printAll = printAll
        self.comment = ""
        if noise == "low":
            self.fluxValidate = fluxValidate + torch.load("lowValidationNoise.pt")
        elif noise == "mid":
            self.fluxValidate = fluxValidate + torch.load("midValidationNoise.pt")
        elif noise == "high":
            self.fluxValidate = fluxValidate + torch.load("highValidationNoise.pt")
        self.noise = noise
        

    
    
    def randomSample(self, num, noise="low", source="train"):
        if source == "test":
            tSet = tauTest
            fSet = fluxTest
        elif source == "validate":
            tSet = tauValidate
            fSet = fluxValidate
        elif source == "train":
            tSet = tauTrain
            fSet = fluxTrain
        else:
            print(source)
            print("Not valid")



        indices = torch.randint(low=0,high=tSet.shape[0],size=(num,))
        points = torch.randint(low=0,high=512,size=(num,))
        taus = torch.zeros((num))
        fluxs = torch.zeros(num,512)
        taus = tSet[indices,0,points]
        fluxs = fSet[indices, 0]
        points = points-256
        scatterPoints = torch.zeros(num, 512, dtype=torch.long).cuda()
        scatterPoints = torch.add(scatterPoints, torch.arange(512))
        scatterPoints = torch.add(scatterPoints, points.view(num,1))
        scatterPoints[scatterPoints < 0] += 512
        scatterPoints[scatterPoints > 511] -= 512
        fluxs = fluxs.gather(1, scatterPoints)

        noiseGen =  torch.distributions.normal.Normal(0.0, signalRMS*noiseDict[noise], validate_args=None)

        fluxs = fluxs + noiseGen.sample(fluxs.shape)
        return taus, fluxs
    
    
    def validate(self, noise):
        loss_func = torch.nn.MSELoss()
        loss = torch.zeros(512)
        self.net.eval()
        with torch.no_grad():
            for midpoint in range(512):
                fluxs = torch.cat((self.fluxValidate[:,:,midpoint-256:],self.fluxValidate[:,:,:midpoint-256]),2)
                taus = tauValidate[:,0,midpoint]
                prediction = self.net(fluxs)
                loss[midpoint] = loss_func(prediction, taus)
        self.net.train()
        return loss.mean()
    
    
    def run(self, epochs, learningRate):
        noise = self.noise
        now = datetime.now()
 
        print("now =", now)
    
        
        loss_func = torch.nn.MSELoss()
        sampler = self.randomSample
        
        pathString = "ConvNN_default_" + str(epochs) + "_" + str(learningRate) + "_" + noise + "_" + \
                   now.strftime('%Y-%m-%d_%H-%M-%S')
        
        writer = SummaryWriter(pathString)
        
        writer.add_text("Net", str(self.net))
        writer.add_text("Epochs", str(epochs))
        writer.add_text("Learning Rate", str(learningRate))
        writer.add_text("Comment", self.comment)
        
        optimizer = torch.optim.Adam( self.net.parameters(), learningRate, weight_decay=0.0005 )
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, gamma=1)
        
        
        validationLoss = self.validate(noise)
        writer.add_scalar('Loss/validation', validationLoss, 0)
        
        self.results(noise, 0, writer)
            
        for epoch in range(epochs):
            
            
            tauSet,fluxSet = sampler(num=10000, noise=noise, source="train")
            tauSet = tauSet.float().cuda()
            fluxSet = fluxSet.float().cuda()
            tauSet = torch.reshape(tauSet, (-1,1,1))
            fluxSet = torch.reshape(fluxSet, (-1,1,512))
            
            prediction = self.net(fluxSet)
                
            
            loss = loss_func(prediction, tauSet)     # must be (1. nn output, 2. target)
            optimizer.zero_grad()   # clear gradients for next train
            loss.backward()         # backpropagation, compute gradients
            optimizer.step()        # apply gradients
            scheduler.step()        # scheduler decreases learning rate geometrically every n epochs

            
            
            
            writer.add_scalar('Loss/train', loss, epoch)
                        
            
            
            
            if self.printAll:
                if epoch % 100 == 0:
                    print("Epoch = ", epoch)
            if epoch % 1000 == 999:
                hundredth = plt.figure()
                a = torch.flatten(tauSet)
                b = torch.flatten(prediction)
                plt.title("%dth epoch predictions" %epoch)
                plt.plot(a.cpu().detach().numpy(),"r", label="actual")
                plt.plot(b.cpu().detach().numpy(),"g.", label="prediction")
                plt.show()
                writer.add_figure("Every Thousand Epochs/" + str(epoch), hundredth)
                
                validationLoss = self.validate(noise)
                writer.add_scalar('Loss/validation', validationLoss, epoch)
                self.results(noise, epoch, writer)
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.net.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss
                    }, "CopyStorage/" + pathString + "Copy" + str(epoch))


        self.results(noise, epoch, writer)
        
        end = datetime.now()
 
        print("end =", end)
        
        return 1
        
    def results(self, noise, epoch, writer):
        taus = tauValidate[0,0,...].cuda()
        fluxs = torch.zeros((512,1,512)).cuda()
        for i in range(512):
            fluxs[i] = spin(self.fluxValidate[0,0,...],i)
    
        fluxs = fluxs
        
        
        taus = taus.float()
        fluxs = fluxs.float()
        taus = torch.reshape(taus, (-1,1,1))
        fluxs = torch.reshape(fluxs, (-1,1,512))
        prediction = self.net(fluxs)
        prediction = torch.flatten(prediction)
        taus = torch.flatten(taus)
        logPrediction = log_predictor(torch.flatten(self.fluxValidate[0,0,...]))
                                       
        example = plt.figure()
        
        plt.plot(taus.data.cpu().numpy(), "r", label="actual")
        plt.plot(prediction.data.cpu().numpy(), "g", label="prediction")
        plt.plot(logPrediction, "b", label="log and cubic splines")
        plt.legend()
        plt.show()
        writer.add_figure("One Sightline/" + str(epoch), example)
        
        difference = plt.figure()
        plt.plot(prediction.data.cpu().numpy()-taus.data.cpu().numpy(), "g", label="prediction")
        plt.plot(logPrediction-taus.data.cpu().numpy(), "b", label="log and cubic splines")
        plt.legend()
        plt.title("Difference")
        plt.show()
        writer.add_figure("Difference/" + str(epoch), difference)
        
        fracDifference = plt.figure()
        plt.plot((prediction.data.cpu().numpy()-taus.data.cpu().numpy())/taus.data.cpu().numpy(), "g", label="prediction")
        plt.plot((logPrediction-taus.data.cpu().numpy())/taus.data.cpu().numpy(), "b", label="log and cubic splines")
        plt.legend()
        plt.title("Fractional Difference")
        plt.show()
        writer.add_figure("Fractional Difference/" + str(epoch), fracDifference)
        
        predictionList = np.zeros((512*20))
        with torch.no_grad():
            for midpoint in range(512):
                fluxs = torch.cat((self.fluxValidate[0:20,:,midpoint-256:],self.fluxValidate[0:20,:,:midpoint-256]),2)
                prediction = self.net(fluxs)
                predictionList[midpoint::512] = prediction.flatten().data.cpu().numpy()
        self.net.train()
        
        logPList = np.zeros((512*20))
        for i in range(20):
            logP = log_predictor(torch.flatten(self.fluxValidate[i,0]))
            logPList[i*512:(i+1)*512] = logP
            
        
        a = torch.flatten(tauValidate[0:20,0,:]).cpu().numpy()
        b = predictionList
        logP = logPList
        pearsonCoeff = pearsonr(a,b)
        RMSD = RMSE(b, a)
        writer.add_scalar("NeuralNet/Total/RMSD", RMSD, epoch)
        logRMSD = RMSE(logP, a)
        writer.add_scalar("LogPrediction/Total/RMSD", logRMSD, epoch)
        
        print("Pearson's Coefficient is:")
        print(pearsonCoeff)
        writer.add_scalar("NeuralNet/Total/Pearson", pearsonCoeff[0], epoch)
        print("Compared to log:")
        logPearson = pearsonr(a,logP)
        print(logPearson)
        writer.add_scalar("LogPrediction/Total/Pearson", logPearson[0], epoch)
        
        c = np.zeros(a.shape)
        
        newA = np.where(a>2, a, c)
        newB = np.where(a>2, b, c)
        newLogP = np.where(a>2, logP, c)
        newA = newA[np.nonzero(newA)]
        newB = newB[np.nonzero(newB)]
        highA = np.copy(newA)
        highB = np.copy(newB)
        newLogP = newLogP[np.nonzero(newLogP)]
        RMSD = RMSE(newB, newA)
        writer.add_scalar("NeuralNet/High/RMSD", RMSD, epoch)
        print("Pearson's Coefficient for high numbers")
        pearsonCoeff = pearsonr(newA,newB)
        print(pearsonCoeff)
        writer.add_scalar("NeuralNet/High/Pearson", pearsonCoeff[0], epoch)
        print("Compared to log:")
        logPearson = pearsonr(newA,newLogP)
        print(logPearson)
        writer.add_scalar("LogPrediction/High/Pearson", logPearson[0], epoch)
        logRMSD = RMSE(newLogP, newA)
        writer.add_scalar("LogPrediction/High/RMSD", logRMSD, epoch)
        
        
        print("percent high:")
        print(len(newA)/len(a))
        
        newA = np.where(a<2, a, c)
        newB = np.where(a<2, b, c)
        newLogP = np.where(a<2, logP, c)
        newA = newA[np.nonzero(newA)]
        newB = newB[np.nonzero(newB)]
        newLogP = newLogP[np.nonzero(newLogP)]
        RMSD = RMSE(newB, newA)
        writer.add_scalar("NeuralNet/Low/RMSD", RMSD, epoch)
        print("Pearson's Coefficient for low numbers")
        pearsonCoeff = pearsonr(newA,newB)
        print(pearsonCoeff)
        writer.add_scalar("NeuralNet/Low/Pearson", pearsonCoeff[0], epoch)
        print("Compared to log:")
        logPearson = pearsonr(newA,newLogP)
        print(logPearson)
        writer.add_scalar("LogPrediction/Low/Pearson", logPearson[0], epoch)
        logRMSD = RMSE(newLogP, newA)
        writer.add_scalar("LogPrediction/Low/RMSD", logRMSD, epoch)
        
        print("percent low:")
        print(len(newA)/len(a))
        
        line = np.linspace(0,a.max())
        scatter = plt.figure()
        plt.scatter(a,b)
        plt.title("Scatter plot of Prediction vs Tau")
        plt.ylabel("Prediction")
        plt.xlabel("Tau")
        plt.plot(line, line, "k")
        plt.show()
        writer.add_figure("Scatter/" + str(epoch), scatter)

IndentationError: expected an indented block (<ipython-input-1-34ed99248baa>, line 37)